# Evals for multi-turn dialogue using Inspect

This notebook illustrates how to use [AISI's Inspect open source framework](https://ukgovernmentbeis.github.io/inspect_ai/) to evaluate LLM's in multi-turn dialogues.

I use example evals from MT Bench, copied from the [fastchat github repo](https://github.com/lm-sys/FastChat/blob/main/fastchat/llm_judge/data/mt_bench/question.jsonl) and stored in `question.jsonl`.

I use models from OpenAI.
It should be easy to adjust this example for other models.

Contents
- How to run the notebook
- Create Inspect `Dataset`
- Create `solver`
- Create `scorer`
- Run the evals

## How to run the notebook

1. Locally clone this repository.
1. Create a virtual environment with python and install the relevant packages by running:
`pip install inspect-ai openai ipykernel ipywidgets
`
1. Create a `.env` file in the root of this repository with one line:
`
OPENAI_API_KEY=your-api-key
`

In [1]:
import re

from inspect_ai.model import ChatMessageUser, ChatMessageSystem, Model, get_model, ChatMessage, ChatMessageAssistant
from inspect_ai.dataset import Sample, json_dataset
from inspect_ai.solver import Generate, Solver, TaskState, solver
from inspect_ai.scorer import Scorer, Score, scorer, INCORRECT, Target, mean, accuracy, bootstrap_std
from inspect_ai import Task, eval, task

## Create Inspect `Dataset`

Essentially, a `Dataset` is a collection of `Sample` objects.
For multi-turn dialogues, a `Sample` consists of a list of user questions that will be asked one after the other in a single dialogue.
See a couple of examples below.

Note that MT Bench only has two questions per sample, but in principle you can have as many as you wish.

In [2]:
# A 'record' is a dictionary corresponding to one line from the jsonl file.
# In particular, it has a key 'turns' that is a list of strings.
def record_to_sample(record):
    return Sample(
        input = [ChatMessageUser(content=turn) for turn in record['turns']],
        id=record['question_id']
    )

dataset = json_dataset("question.jsonl", record_to_sample)

In [3]:
for single_input in dataset[0].input:
    print(single_input)

Sample(input=[ChatMessageUser(content='Compose an engaging travel blog post about a recent trip to Hawaii, highlighting cultural experiences and must-see attractions.', source=None, role='user'), ChatMessageUser(content='Rewrite your previous response. Start every sentence with the letter A.', source=None, role='user')], choices=None, target='', id=81, metadata=None)

In [4]:
dataset[-1]

Sample(input=[ChatMessageUser(content='Suggest five award-winning documentary films with brief background descriptions for aspiring filmmakers to study.', source=None, role='user'), ChatMessageUser(content='With the spirit in the first film, craft a succinct and persuasive pitch for a film about overcoming adversity.', source=None, role='user')], choices=None, target='', id=160, metadata=None)

## Create `solver`

Solvers are essentially the scaffolding around an LLM to process a single Sample from the dataset.

In [5]:
@solver
def multi_dialogue_solver() -> Solver:
    """Solver for the multi-dialogue task.

    This solver simulates a dialogue between a user and the LLM.

    It assumes that `state.messages` is a list of ChatMessageUser objects.
    It will write to `state.messages` a list of ChatMessageUser and ChatMessageSystem
    objects corresponding to the dialogue between the user and the LLM.
    """
    async def solve(state: TaskState, generate: Generate) -> TaskState:
        # copy the messages from the state
        input = state.messages.copy()

        # for this solver, we expect input to be a list of ChatMessageUser
        if not isinstance(input, list):
            raise TypeError(f'Inputs in samples of the dataset should be list of ChatMessageUser. Found {input}')
        if not all(isinstance(turn, ChatMessageUser) for turn in input):
            raise TypeError(f'Inputs in samples of the dataset should be list of ChatMessageUser. Found {input}')

        # generate the output after each turn in the input and write to state.messages
        state.messages = []
        for turn in input:
            state.messages.append(turn)
            state = await generate(state)

        return state

    return solve

## Create `scorer`

`model_graded_dialogue`. Scorer that evaluates the LLM's output (created from the solver) by using another LLM.
Based on the prompt from MT Bench. 

In [6]:
# prompt based on 'single-v1-multi-turn' prompt from https://github.com/lm-sys/FastChat/blob/main/fastchat/llm_judge/data/judge_prompts.jsonl
# Changed system prompt so it judges all answers to user questions, not just answer to the second question.
# Changed prompt template to remove reference to assistant A and to allow for conversations of arbitrary length.

SYSTEM_PROMPT = "Please act as an impartial judge and evaluate the quality of the response provided by an AI assistant to the user question displayed below. Your evaluation should consider factors such as the helpfulness, relevance, accuracy, depth, creativity, and level of detail of the response. You evaluation should focus on all the assistant's answers to the user's questions. Begin your evaluation by providing a short explanation. Be as objective as possible. After providing your explanation, you must rate the response on a scale of 1 to 10 by strictly following this format: \"[[rating]]\", for example: \"Rating: [[5]]\".\n\n"
PROMPT_TEMPLATE_START = "<|The Start of the Assistant's Conversation with User|>\n\n"
PROMPT_TEMPLATE_USER = "### User:\n{message}\n\n"
PROMPT_TEMPLATE_ASSISTANT = "### Assistant:\n{message}\n\n"
PROMPT_TEMPLATE_END = "<|The End of the Assistant's Conversation with User|>"

# scorer adapted from example of model_graded_qa from https://ukgovernmentbeis.github.io/inspect_ai/scorers.html

@scorer(metrics=[mean(), bootstrap_std()])
def model_graded_dialogue(
    system_prompt: str = SYSTEM_PROMPT,
    prompt_template_start: str = PROMPT_TEMPLATE_START,
    prompt_template_user: str = PROMPT_TEMPLATE_USER,
    prompt_template_assistant: str = PROMPT_TEMPLATE_ASSISTANT,
    prompt_template_end: str = PROMPT_TEMPLATE_END,
    model: str | Model | None = None,
) -> Scorer:
    # resolve model
    grader_model = get_model(model)

    async def score(state: TaskState, target: Target) -> Score:
        # create system message
        system_message = ChatMessageSystem(content=system_prompt)

        # create user message, by looping through TaskState.messages
        user_message = prompt_template_start
        for message in state.messages:
            if isinstance(message, ChatMessageUser):
                user_message += prompt_template_user.format(message=message.content)
            elif isinstance(message, ChatMessageAssistant):
                user_message += prompt_template_assistant.format(message=message.content)
            else:
                continue
        user_message += prompt_template_end

        user_message = ChatMessageUser(content=user_message)

        # query the model for the score
        result = await grader_model.generate([system_message, user_message])

        # extract the grade
        grade_pattern = r"Rating: \[\[(\d+)\]\]"
        match = re.search(grade_pattern, result.completion)
        if match:
            return Score(
                value=int(match.group(1)),
                explanation=result.completion,
            )
        else:
            return Score(
                value=0,
                explanation="Grade not found in model output: "
                + f"{result.completion}",
            )

    return score

## Run the evals

After running an eval, a log will be created as a json file in `./logs/`.
You can view the (latest) log in a user friendly GUI by running `inspect view` in terminal.

In [ ]:
@task
def multi_dialogue_task():
    return Task(
        dataset=dataset,
        plan=[
          multi_dialogue_solver(),
        ],
        scorer=model_graded_dialogue(model="openai/gpt-3.5-turbo")
    )

logs = eval(
    tasks=multi_dialogue_task(),
    model="openai/gpt-3.5-turbo",
)